In [28]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys
import sqlite3
import torch

sys.path.insert(0, '../src')
sys.path.insert(1, '..')

from main import get_model
from models.config import Config
import db_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
sys.argv = ['notebooks/get_representations.ipynb',
            '--config', '../configs/models/qwen/Qwen2-VL-2B-Instruct-VisualFlow.yaml']

config = Config()

In [4]:
model = get_model(config.architecture, config)
# print(model.model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.29it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [75]:
print(model.model)

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

In [13]:
model.run(save_tokens=True)

Running forward hooks on data:   0%|          | 0/2 [00:00<?, ?it/s]

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151

Running forward hooks on data:  50%|█████     | 1/2 [00:00<00:00,  2.75it/s]

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151

Running forward hooks on data: 100%|██████████| 2/2 [00:00<00:00,  2.68it/s]


In [14]:
db_path = model.config.output_db
print(db_path)
connection = sqlite3.connect(db_path)
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

../output/Qwen2-VL-2B-Instruct-VisualFlow.db


In [16]:
for r in cursor.execute("""
  SELECT id, image_id, name, image_path, prompt, label, layer, pooling_method, tensor_dim, tensor_shape,
         length(tensor) AS tensor_bytes
  FROM tensors
  LIMIT 20
"""):
    print(dict(r))

{'id': 1, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '../data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'input_ids', 'pooling_method': None, 'tensor_dim': 1, 'tensor_shape': 'torch.Size([1, 129])', 'tensor_bytes': 2204}
{'id': 2, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'visual.blocks.31', 'pooling_method': None, 'tensor_dim': 1280, 'tensor_shape': 'torch.Size([392, 1280])', 'tensor_bytes': 1004700}
{'id': 3, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'visual.merger', 'pooling_method': None, 'tensor_dim': 1536, 'tensor_shape': 'torch.Size([98,

In [ ]:
for r in cursor.execute("""
  SELECT id, name, image_path, layer, tensor_dim, tensor_shape,
         length(tensor) AS tensor_bytes
  FROM tensors
  WHERE layer = "input_ids"
  LIMIT 20
"""):
    print(dict(r))

{'id': 1, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '../data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'input_ids', 'pooling_method': None, 'tensor_dim': 1, 'tensor_shape': 'torch.Size([1, 129])', 'tensor_bytes': 2204}
{'id': 7, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '../data/test-images/black_in_black.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'input_ids', 'pooling_method': None, 'tensor_dim': 1, 'tensor_shape': 'torch.Size([1, 129])', 'tensor_bytes': 2204}
{'id': 13, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'input_ids', 'pooling_method': None, 'tensor_dim': 1, 'tensor_shape': 'torch.Size([1, 129])', 'tensor_bytes': 2204}
{'id': 19, 'imag

In [29]:
embeddings = db_utils.get_embeddings_by_layer(
    db_path=db_path,
    layer_name="input_ids",
    device="cuda"
)

print(type(embeddings[0]))
print(embeddings[0])
embeddings = tuple(map(list, zip(*embeddings)))[1]

print(torch.tensor(np.array(embeddings)).shape)

<class 'tuple'>
('input_ids', array([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,
            13, 151645,    198, 151644,    872,    198, 151652, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
        151655, 151655, 151655, 15